# Build a first version of the event selection for $H \rightarrow 4 \ell$

This will test Coffea to see if we can figure out how to use it with our code.

First are the includes from coffea. This is based on the [example written by Ben](https://github.com/CoffeaTeam/coffea/blob/master/binder/servicex/ATLAS/LocalExample.ipynb).

In [41]:
from servicex import ServiceXDataset
from coffea.processor.servicex import DataSource, FuncAdlDataset, Analysis
from coffea.processor.servicex import LocalExecutor 

import matplotlib.pyplot as plt
import numpy as np

from coffea import hist, processor
from IPython.display import display, update_display, HTML

And imports connected with running servicex.

In [42]:
from func_adl import ObjectStream
from func_adl_servicex import ServiceXSourceUpROOT
from hist import Hist
import mplhep as mpl
import awkward as ak

from utils import files

Methods copied to help us get all leptons from the source files

In [43]:
def apply_event_cuts (source: ObjectStream) -> ObjectStream:
    '''Event level cuts for the analysis. Keep from sending data that we aren't going to need at all in the end.
    '''
    return (source
        .Where(lambda e: e.trigE or e.trigM))
def good_leptons(source: ObjectStream) -> ObjectStream:
    '''Select out all good leptons from each event. Return their pt, eta, phi, and E, and other
    things needed downstream.

    Because uproot doesn't tie toegher the objects, we can't do any cuts at this point.
    '''
    return source.Select(lambda e:
        {
            'lep_pt': e.lep_pt,
            'lep_eta': e.lep_eta,
            'lep_phi': e.lep_phi,
            'lep_energy': e.lep_E,
            'lep_charge': e.lep_charge,
            'lep_ptcone30': e.lep_ptcone30,
            'lep_etcone20': e.lep_etcone20,
            'lep_typeid': e.lep_type,
            'lep_trackd0pvunbiased': e.lep_trackd0pvunbiased,
            'lep_tracksigd0pvunbiased': e.lep_tracksigd0pvunbiased,
            'lep_z0': e.lep_z0,
        }) \
        .AsParquetFiles('junk.parquet')

Create the `func_adl` cuts to get the data. The dataset we use here doesn't matter, as long as it "looks" like all the datasets we are going to be processing.

In [44]:
ds = ServiceXSourceUpROOT('cernopendata://dummy',  files['ggH125_ZZ4lep']['treename'], backend='open_uproot')
ds.return_qastle = True
leptons = good_leptons(apply_event_cuts(ds))

The analysis code that will apply the 4 lepton cuts and make the 4 lepton mass plot.

In [45]:
class ATLAS_Higgs_4L(Analysis):
    @staticmethod
    def process(events):
        import awkward as ak
        from collections import defaultdict

        sumw = defaultdict(float)
        # TODO: Add channel so we can see the different shapes
        # But plot1d seems to fail here
        mass_hist = hist.Hist(
            "Events",
            hist.Cat("dataset", "Dataset"),
            hist.Bin("mass", "$Z_{ee}$ [GeV]", 120, 60, 180),
        )

        dataset = events.metadata['dataset']
        leptons = events.lep

        # Good electon selection
        electrons_mask = ((leptons.typeid == 11)
            & (leptons.pt > 7000.0)
            & (abs(leptons.eta) <2.47)
            & (leptons.etcone20 / leptons.pt < 0.3)
            & (leptons.ptcone30 / leptons.pt < 0.3)
            & (abs(leptons.trackd0pvunbiased) / leptons.tracksigd0pvunbiased < 5)
            & (abs(leptons.z0*np.sin(leptons.theta)) < 0.5)
        )

        electrons_good = leptons[electrons_mask]

        # Good muon selection
        muon_mask = ((leptons.typeid == 11)
            & (leptons.pt > 5000.0)
            & (abs(leptons.eta) <2.5)
            & (leptons.etcone20 / leptons.pt < 0.3)
            & (leptons.ptcone30 / leptons.pt < 0.3)
            & (abs(leptons.trackd0pvunbiased) / leptons.tracksigd0pvunbiased < 3)
            & (abs(leptons.z0*np.sin(leptons.theta)) < 0.5)
        )

        muons_good = leptons[leptons.typeid == 13]

        # In order to cut in sorted lepton pt, we have to rebuild a lepton array here
        leptons_good = ak.concatenate((electrons_good, muons_good), axis=1)
        leptons_good_index = ak.argsort(leptons_good.pt, ascending=False)
        leptons_good_sorted = leptons_good[leptons_good_index]

        # Event level cuts now that we know the good leptons
        # - We need to look at 4 good lepton events only
        # - We need same flavor, so check for even numbers of each flavor
        # - all charges must be balenced
        event_mask = (
            (ak.num(leptons_good_sorted) == 4)
            & ((ak.num(electrons_good) == 0) | (ak.num(electrons_good) == 2) | (ak.num(electrons_good) == 4))
            & ((ak.num(muons_good) == 0) | (ak.num(muons_good) == 2) | (ak.num(muons_good) == 4))
            & (ak.sum(electrons_good.charge, axis=1) == 0)
            & (ak.sum(muons_good.charge, axis=1) == 0)
        )

        # Next, we need to cut on the pT for the leading, sub-leading, and sub-sub-leading lepton
        leptons_good_preselection = leptons_good[event_mask]
        event_good_lepton_mask = (
            (leptons_good_preselection[:,0].pt > 25000.0)
            & (leptons_good_preselection[:,1].pt > 15000.0)
            & (leptons_good_preselection[:,2].pt > 10000.0)
        )

        # Now, we need to rebuild the good muon and electron lists with those selections
        muons_analysis = muons_good[event_mask][event_good_lepton_mask]
        electrons_analysis = electrons_good[event_mask][event_good_lepton_mask]

        # Lets do eemumu events - as there are no permutations there.abs
        # At this point if there are two muons, there must be two electrons
        eemumu_mask = (ak.num(muons_analysis) == 2)
        muon_eemumu = muons_analysis[eemumu_mask]
        electrons_eemumu = electrons_analysis[eemumu_mask]
        z1_eemumu = muon_eemumu[:,0] + muon_eemumu[:,1]
        z2_eemumu = electrons_eemumu[:,0] + electrons_eemumu[:,1]
        h_eemumu = z1_eemumu + z2_eemumu

        sumw[dataset] += len(h_eemumu)
        mass_hist.fill(
            dataset=dataset,
            mass=h_eemumu.mass/1000.0,
        )
        
        return {
            "sumw": sumw,
            "mass": mass_hist,
        }

Create the data source that we will be running against.

In [46]:
def make_ds(name: str, query: ObjectStream):
    '''Create a ServiceX Datasource for a particular ATLAS Open data file
    '''
    datasets = [ServiceXDataset(files[name]['files'], backend_type='open_uproot')]
    return DataSource(query=query, metadata={'dataset': name}, datasets=datasets)

And run!

In [47]:
analysis = ATLAS_Higgs_4L()
# TODO: It would be good if datatype was determined automagically (there is enough info)
executor = LocalExecutor(datatype='parquet')
datasource = make_ds('ggH125_ZZ4lep', leptons)

async def run_updates_stream(accumulator_stream):
  global first

  count = 0
  async for coffea_info in accumulator_stream:
    count += 1
    print(count, coffea_info)
  return coffea_info

# Why do I need run_updates_stream, why not just await on execute (which fails with async gen can't).
# Perhaps something from aiostream can help here?
result = await run_updates_stream(executor.execute(analysis, datasource))

UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype([('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('t', '<f4')]), dtype('int32')) -> None

In [40]:
hist.plot1d(result['mass'])

ValueError: plot1d() can only support up to two dimensions (one for axis, one to stack or overlay)